## Read and Clean Dataset


Import the libraries.


In [1]:
# Install tensorflow and keras in Anaconda
# !pip install tensorflow
# !pip install keras

In [2]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn import metrics

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Read the data into a <em>pandas</em> dataframe.


In [3]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.


#### Let's check how many data points we have.


In [4]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors (X) and target (y)


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [7]:
concrete_data_columns = concrete_data.columns

X = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
y = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [8]:
X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [10]:
X_norm = (X - X.mean()) / X.std()
X_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [11]:
n_cols = X_norm.shape[1] # number of predictors
print("Number of features:", n_cols)

Number of features: 8


<a id="item1"></a>


<a id='item32'></a>


## Build a Neural Network


Let's define a function that defines our neural network model for us so that we can conveniently call it to create our model.


In [12]:
# define regression model
def create_neural_network():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has one hidden layers, each of 10 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [13]:
# build the model
model = create_neural_network()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [14]:
# fit the model
mean_sq_error = [] # Empty list of mean squared error

def split_and_fit_model(myModel):
    X_train, X_test, y_train, y_test = train_test_split( X_norm, y, test_size=0.3, random_state=4)
    
    myModel.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=2)
    
    # evaluate the model
    scores = myModel.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: {} \n Error: {}".format(scores, 100-scores*100))
    
    # Get the predicted values
    yhat = myModel.predict(X_test)
    mse_sk = metrics.mean_squared_error(y_test, yhat) # Calculate RMSE
    mean_sq_error.append(mse_sk)

# Step 4
for i in range(50):
    print("Iteration %s:\n" %str(i))
    split_and_fit_model(model)
    
print(len(mean_sq_error))
print("Mean:", np.mean(np.array(mean_sq_error)))
print("Standard Deviation :", np.std(np.array(mean_sq_error)))

Iteration 0:

Epoch 1/50
16/16 - 1s - loss: 1609.2684 - val_loss: 1495.4851 - 547ms/epoch - 34ms/step
Epoch 2/50
16/16 - 0s - loss: 1598.9500 - val_loss: 1485.8910 - 37ms/epoch - 2ms/step
Epoch 3/50
16/16 - 0s - loss: 1589.0752 - val_loss: 1476.0609 - 36ms/epoch - 2ms/step
Epoch 4/50
16/16 - 0s - loss: 1578.8438 - val_loss: 1466.4382 - 36ms/epoch - 2ms/step
Epoch 5/50
16/16 - 0s - loss: 1568.6809 - val_loss: 1456.6890 - 36ms/epoch - 2ms/step
Epoch 6/50
16/16 - 0s - loss: 1558.3590 - val_loss: 1446.7198 - 33ms/epoch - 2ms/step
Epoch 7/50
16/16 - 0s - loss: 1547.8524 - val_loss: 1436.3928 - 38ms/epoch - 2ms/step
Epoch 8/50
16/16 - 0s - loss: 1536.9347 - val_loss: 1425.9480 - 37ms/epoch - 2ms/step
Epoch 9/50
16/16 - 0s - loss: 1525.8269 - val_loss: 1414.9755 - 37ms/epoch - 2ms/step
Epoch 10/50
16/16 - 0s - loss: 1514.3022 - val_loss: 1403.4508 - 41ms/epoch - 3ms/step
Epoch 11/50
16/16 - 0s - loss: 1501.9120 - val_loss: 1391.7264 - 39ms/epoch - 2ms/step
Epoch 12/50
16/16 - 0s - loss: 1489.

Epoch 46/50
16/16 - 0s - loss: 215.4165 - val_loss: 213.1722 - 39ms/epoch - 2ms/step
Epoch 47/50
16/16 - 0s - loss: 213.0199 - val_loss: 211.4557 - 39ms/epoch - 2ms/step
Epoch 48/50
16/16 - 0s - loss: 210.7116 - val_loss: 209.8757 - 33ms/epoch - 2ms/step
Epoch 49/50
16/16 - 0s - loss: 208.5272 - val_loss: 208.3898 - 36ms/epoch - 2ms/step
Epoch 50/50
16/16 - 0s - loss: 206.5018 - val_loss: 206.9209 - 40ms/epoch - 2ms/step
Accuracy: 219.24423217773438 
 Error: -21824.423217773438
10/10 [==============================] - 0s 1ms/step
Iteration 2:

Epoch 1/50
16/16 - 0s - loss: 204.5346 - val_loss: 205.5136 - 97ms/epoch - 6ms/step
Epoch 2/50
16/16 - 0s - loss: 202.6102 - val_loss: 204.1789 - 44ms/epoch - 3ms/step
Epoch 3/50
16/16 - 0s - loss: 200.7099 - val_loss: 202.9351 - 44ms/epoch - 3ms/step
Epoch 4/50
16/16 - 0s - loss: 199.0450 - val_loss: 201.7133 - 48ms/epoch - 3ms/step
Epoch 5/50
16/16 - 0s - loss: 197.2636 - val_loss: 200.5743 - 39ms/epoch - 2ms/step
Epoch 6/50
16/16 - 0s - loss: 

Epoch 40/50
16/16 - 0s - loss: 116.4348 - val_loss: 131.4013 - 34ms/epoch - 2ms/step
Epoch 41/50
16/16 - 0s - loss: 115.6589 - val_loss: 130.7852 - 36ms/epoch - 2ms/step
Epoch 42/50
16/16 - 0s - loss: 115.1114 - val_loss: 130.1643 - 34ms/epoch - 2ms/step
Epoch 43/50
16/16 - 0s - loss: 114.4349 - val_loss: 129.4929 - 34ms/epoch - 2ms/step
Epoch 44/50
16/16 - 0s - loss: 113.7946 - val_loss: 128.7530 - 35ms/epoch - 2ms/step
Epoch 45/50
16/16 - 0s - loss: 113.1553 - val_loss: 128.1623 - 40ms/epoch - 2ms/step
Epoch 46/50
16/16 - 0s - loss: 112.6025 - val_loss: 127.4022 - 37ms/epoch - 2ms/step
Epoch 47/50
16/16 - 0s - loss: 111.9506 - val_loss: 126.9852 - 33ms/epoch - 2ms/step
Epoch 48/50
16/16 - 0s - loss: 111.2987 - val_loss: 126.2432 - 35ms/epoch - 2ms/step
Epoch 49/50
16/16 - 0s - loss: 110.6785 - val_loss: 125.6926 - 37ms/epoch - 2ms/step
Epoch 50/50
16/16 - 0s - loss: 110.0825 - val_loss: 125.1395 - 33ms/epoch - 2ms/step
Accuracy: 132.43576049804688 
 Error: -13143.576049804688
10/10 [

Epoch 35/50
16/16 - 0s - loss: 74.6438 - val_loss: 88.8694 - 40ms/epoch - 2ms/step
Epoch 36/50
16/16 - 0s - loss: 74.3427 - val_loss: 88.6593 - 40ms/epoch - 3ms/step
Epoch 37/50
16/16 - 0s - loss: 74.0644 - val_loss: 88.2970 - 36ms/epoch - 2ms/step
Epoch 38/50
16/16 - 0s - loss: 73.8054 - val_loss: 88.0948 - 37ms/epoch - 2ms/step
Epoch 39/50
16/16 - 0s - loss: 73.5644 - val_loss: 87.8985 - 43ms/epoch - 3ms/step
Epoch 40/50
16/16 - 0s - loss: 73.3078 - val_loss: 87.5670 - 51ms/epoch - 3ms/step
Epoch 41/50
16/16 - 0s - loss: 73.0206 - val_loss: 87.2396 - 40ms/epoch - 2ms/step
Epoch 42/50
16/16 - 0s - loss: 72.8374 - val_loss: 86.9720 - 40ms/epoch - 2ms/step
Epoch 43/50
16/16 - 0s - loss: 72.5428 - val_loss: 86.7345 - 36ms/epoch - 2ms/step
Epoch 44/50
16/16 - 0s - loss: 72.3363 - val_loss: 86.5193 - 37ms/epoch - 2ms/step
Epoch 45/50
16/16 - 0s - loss: 72.0425 - val_loss: 86.3412 - 32ms/epoch - 2ms/step
Epoch 46/50
16/16 - 0s - loss: 71.8056 - val_loss: 86.0951 - 35ms/epoch - 2ms/step
Epoc

Epoch 31/50
16/16 - 0s - loss: 56.5158 - val_loss: 69.2262 - 35ms/epoch - 2ms/step
Epoch 32/50
16/16 - 0s - loss: 56.3394 - val_loss: 68.9460 - 33ms/epoch - 2ms/step
Epoch 33/50
16/16 - 0s - loss: 56.2249 - val_loss: 68.7839 - 35ms/epoch - 2ms/step
Epoch 34/50
16/16 - 0s - loss: 56.1177 - val_loss: 68.6239 - 34ms/epoch - 2ms/step
Epoch 35/50
16/16 - 0s - loss: 55.9256 - val_loss: 68.4528 - 38ms/epoch - 2ms/step
Epoch 36/50
16/16 - 0s - loss: 55.7679 - val_loss: 68.2781 - 38ms/epoch - 2ms/step
Epoch 37/50
16/16 - 0s - loss: 55.7945 - val_loss: 68.0413 - 37ms/epoch - 2ms/step
Epoch 38/50
16/16 - 0s - loss: 55.5423 - val_loss: 68.0405 - 35ms/epoch - 2ms/step
Epoch 39/50
16/16 - 0s - loss: 55.3785 - val_loss: 67.8666 - 34ms/epoch - 2ms/step
Epoch 40/50
16/16 - 0s - loss: 55.1912 - val_loss: 67.7196 - 36ms/epoch - 2ms/step
Epoch 41/50
16/16 - 0s - loss: 55.0724 - val_loss: 67.3908 - 34ms/epoch - 2ms/step
Epoch 42/50
16/16 - 0s - loss: 54.9148 - val_loss: 67.2967 - 34ms/epoch - 2ms/step
Epoc

...

Epoch 11/50
16/16 - 0s - loss: 26.7813 - val_loss: 44.0267 - 36ms/epoch - 2ms/step
Epoch 12/50
16/16 - 0s - loss: 26.8323 - val_loss: 44.0199 - 36ms/epoch - 2ms/step
Epoch 13/50
16/16 - 0s - loss: 26.8140 - val_loss: 44.0537 - 40ms/epoch - 2ms/step
Epoch 14/50
16/16 - 0s - loss: 26.7888 - val_loss: 44.0352 - 76ms/epoch - 5ms/step
Epoch 15/50
16/16 - 0s - loss: 26.8345 - val_loss: 44.0135 - 40ms/epoch - 3ms/step
Epoch 16/50
16/16 - 0s - loss: 26.7782 - val_loss: 44.0166 - 34ms/epoch - 2ms/step
Epoch 17/50
16/16 - 0s - loss: 26.8507 - val_loss: 43.9410 - 39ms/epoch - 2ms/step
Epoch 18/50
16/16 - 0s - loss: 26.8073 - val_loss: 44.0288 - 36ms/epoch - 2ms/step
Epoch 19/50
16/16 - 0s - loss: 26.8012 - val_loss: 44.0968 - 34ms/epoch - 2ms/step
Epoch 20/50
16/16 - 0s - loss: 26.8024 - val_loss: 44.0033 - 39ms/epoch - 2ms/step
Epoch 21/50
16/16 - 0s - loss: 26.8185 - val_loss: 44.0589 - 35ms/epoch - 2ms/step
Epoch 22/50
16/16 - 0s - loss: 26.7657 - val_loss: 44.0063 - 36ms/epoch - 2ms/step
Epoc

Epoch 7/50
16/16 - 0s - loss: 26.6814 - val_loss: 43.9558 - 39ms/epoch - 2ms/step
Epoch 8/50
16/16 - 0s - loss: 26.6952 - val_loss: 43.9796 - 33ms/epoch - 2ms/step
Epoch 9/50
16/16 - 0s - loss: 26.6579 - val_loss: 43.9685 - 38ms/epoch - 2ms/step
Epoch 10/50
16/16 - 0s - loss: 26.6998 - val_loss: 43.8655 - 39ms/epoch - 2ms/step
Epoch 11/50
16/16 - 0s - loss: 26.6447 - val_loss: 43.9766 - 39ms/epoch - 2ms/step
Epoch 12/50
16/16 - 0s - loss: 26.7014 - val_loss: 44.0024 - 39ms/epoch - 2ms/step
Epoch 13/50
16/16 - 0s - loss: 26.6601 - val_loss: 43.9866 - 37ms/epoch - 2ms/step
Epoch 14/50
16/16 - 0s - loss: 26.7059 - val_loss: 43.9196 - 39ms/epoch - 2ms/step
Epoch 15/50
16/16 - 0s - loss: 26.6597 - val_loss: 44.0040 - 39ms/epoch - 2ms/step
Epoch 16/50
16/16 - 0s - loss: 26.7037 - val_loss: 43.8777 - 35ms/epoch - 2ms/step
Epoch 17/50
16/16 - 0s - loss: 26.6689 - val_loss: 43.9914 - 38ms/epoch - 2ms/step
Epoch 18/50
16/16 - 0s - loss: 26.6557 - val_loss: 43.9781 - 34ms/epoch - 2ms/step
Epoch 1

Epoch 3/50
16/16 - 0s - loss: 26.5242 - val_loss: 43.8028 - 36ms/epoch - 2ms/step
Epoch 4/50
16/16 - 0s - loss: 26.5511 - val_loss: 43.7951 - 37ms/epoch - 2ms/step
Epoch 5/50
16/16 - 0s - loss: 26.5294 - val_loss: 43.8905 - 35ms/epoch - 2ms/step
Epoch 6/50
16/16 - 0s - loss: 26.5548 - val_loss: 43.9068 - 38ms/epoch - 2ms/step
Epoch 7/50
16/16 - 0s - loss: 26.5330 - val_loss: 43.8919 - 37ms/epoch - 2ms/step
Epoch 8/50
16/16 - 0s - loss: 26.5416 - val_loss: 44.0103 - 40ms/epoch - 2ms/step
Epoch 9/50
16/16 - 0s - loss: 26.5432 - val_loss: 43.8313 - 37ms/epoch - 2ms/step
Epoch 10/50
16/16 - 0s - loss: 26.5325 - val_loss: 43.9009 - 35ms/epoch - 2ms/step
Epoch 11/50
16/16 - 0s - loss: 26.4985 - val_loss: 43.8686 - 49ms/epoch - 3ms/step
Epoch 12/50
16/16 - 0s - loss: 26.5552 - val_loss: 43.9999 - 45ms/epoch - 3ms/step
Epoch 13/50
16/16 - 0s - loss: 26.5335 - val_loss: 43.9256 - 39ms/epoch - 2ms/step
Epoch 14/50
16/16 - 0s - loss: 26.4928 - val_loss: 43.8540 - 36ms/epoch - 2ms/step
Epoch 15/50

Accuracy: 38.224365234375 
 Error: -3722.4365234375
10/10 [==============================] - 0s 889us/step
Iteration 48:

Epoch 1/50
16/16 - 0s - loss: 26.4146 - val_loss: 43.9279 - 80ms/epoch - 5ms/step
Epoch 2/50
16/16 - 0s - loss: 26.3672 - val_loss: 43.9796 - 39ms/epoch - 2ms/step
Epoch 3/50
16/16 - 0s - loss: 26.3793 - val_loss: 43.9697 - 39ms/epoch - 2ms/step
Epoch 4/50
16/16 - 0s - loss: 26.3641 - val_loss: 43.8813 - 39ms/epoch - 2ms/step
Epoch 5/50
16/16 - 0s - loss: 26.3885 - val_loss: 43.8761 - 39ms/epoch - 2ms/step
Epoch 6/50
16/16 - 0s - loss: 26.4406 - val_loss: 43.9123 - 37ms/epoch - 2ms/step
Epoch 7/50
16/16 - 0s - loss: 26.3851 - val_loss: 43.9003 - 33ms/epoch - 2ms/step
Epoch 8/50
16/16 - 0s - loss: 26.3651 - val_loss: 43.9755 - 37ms/epoch - 2ms/step
Epoch 9/50
16/16 - 0s - loss: 26.3681 - val_loss: 43.9652 - 36ms/epoch - 2ms/step
Epoch 10/50
16/16 - 0s - loss: 26.3435 - val_loss: 43.9281 - 36ms/epoch - 2ms/step
Epoch 11/50
16/16 - 0s - loss: 26.3855 - val_loss: 43.845

Epoch 47/50
16/16 - 0s - loss: 26.2704 - val_loss: 43.8198 - 38ms/epoch - 2ms/step
Epoch 48/50
16/16 - 0s - loss: 26.2786 - val_loss: 43.8979 - 52ms/epoch - 3ms/step
Epoch 49/50
16/16 - 0s - loss: 26.2822 - val_loss: 43.7567 - 33ms/epoch - 2ms/step
Epoch 50/50
16/16 - 0s - loss: 26.2508 - val_loss: 43.9423 - 35ms/epoch - 2ms/step
Accuracy: 38.34861373901367 
 Error: -3734.861373901367
10/10 [==============================] - 0s 886us/step
50
Mean: 63.55776134625607
Standard Deviation : 87.02419594306059
